## Model Validation
- Step 0: Training-validation data split
- Step 1: Data analysis
- Step 2: Classification
- **Step 3: Model Validation**
	- Generating a Kaggle submission
	- Analysis of the results

In [1]:
% matplotlib qt

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import modules.utilgen as utgen
import modules.model_framework.model as mdl
import modules.data_framework.data_pipeline as dtp
from modules.utilgen import viewdf
import itertools

% reload_ext autoreload
% autoreload 2

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


D:\SoftwareWork\PythonProjects\Miniconda\envs\datascience1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load data

In [2]:
version = 5
dataPath = os.path.join('data', 'data_derived', 'train.csv')
dataTPath = os.path.join('data', 'data_derived', 'test.csv')
submissionPath = os.path.join('data', 'data_derived', 'submission.csv')
data = pd.read_csv(dataPath, index_col=0)
dataT = pd.read_csv(dataTPath, index_col=0)
with utgen.printOptions(precision=2):
    print('---------- Original Data (Train) -----------')
    display(data.sample(n=5))
    display(data.describe())
    print('---------- Original Data (Test) -----------')
    dataT.insert(loc=0, column='Survived', value=0)
    display(dataT.sample(n=5))
    display(dataT.describe())
    print('---------- Clean Data (Train) -----------')
    data, dataT = dtp.featuresPipelineTrainTest(data, dataT, version=version)
    dataT.Survived = np.nan
    display(data.sample(n=5))
    display(data.describe())
    print('---------- Clean Data (Test) -----------')
    display(dataT.sample(n=5))
    display(dataT.describe())

---------- Original Data (Train) -----------

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
515,0,3,"Coleff, Mr. Satio",male,24.00,0,0,349209,7.50,NaN,S
600,1,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.00,1,0,PC 17485,56.93,A20,C
352,0,1,"Williams-Lambert, Mr. Fletcher Fellows",male,nan,0,0,113510,35.00,C128,S
632,0,3,"Lundahl, Mr. Johan Svensson",male,51.00,0,0,347743,7.05,NaN,S
109,0,3,"Rekic, Mr. Tido",male,38.00,0,0,349249,7.90,NaN,S


,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.00,891.00,714.00,891.00,891.00,891.00
mean,0.38,2.31,29.70,0.52,0.38,32.20
std,0.49,0.84,14.53,1.10,0.81,49.69
min,0.00,1.00,0.42,0.00,0.00,0.00
25%,0.00,2.00,20.12,0.00,0.00,7.91
50%,0.00,3.00,28.00,0.00,0.00,14.45
75%,1.00,3.00,38.00,1.00,0.00,31.00
max,1.00,3.00,80.00,8.00,6.00,512.33


---------- Original Data (Test) -----------

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1276,0,2,"Wheeler, Mr. Edwin Frederick""""",male,nan,0,0,SC/PARIS 2159,12.88,NaN,S
1151,0,3,"Midtsjo, Mr. Karl Albert",male,21.00,0,0,345501,7.78,NaN,S
1225,0,3,"Nakid, Mrs. Said (Waika Mary"" Mowad)""",female,19.00,1,1,2653,15.74,NaN,C
1210,0,3,"Jonsson, Mr. Nils Hilding",male,27.00,0,0,350408,7.85,NaN,S
979,0,3,"Badman, Miss. Emily Louisa",female,18.00,0,0,A/4 31416,8.05,NaN,S


...

,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
PassengerId,,,,,,,,,
367,1,1,60.00,1,0,75.25,1,0,1
757,0,3,28.00,0,0,7.80,0,1,0
386,0,2,18.00,0,0,73.50,0,1,0
650,1,3,23.00,0,0,7.55,0,1,1
448,1,1,34.00,0,0,26.55,0,1,0


,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
count,891.00,891.00,891.00,891.00,891.00,891.00,891.00,891.00,891.00
mean,0.38,2.31,29.60,0.40,0.36,29.74,0.19,0.73,0.35
std,0.49,0.84,13.23,0.64,0.72,35.95,0.39,0.45,0.48
min,0.00,1.00,0.42,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,2.00,21.69,0.00,0.00,7.91,0.00,0.00,0.00
50%,0.00,3.00,28.67,0.00,0.00,14.45,0.00,1.00,0.00
75%,1.00,3.00,37.00,1.00,0.00,31.00,0.00,1.00,1.00
max,1.00,3.00,71.24,2.00,3.00,169.73,1.00,1.00,1.00


---------- Clean Data (Test) -----------

,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
PassengerId,,,,,,,,,
1197,nan,1,64.00,1,1,26.55,0,1,1
1157,nan,3,28.67,0,0,7.90,0,1,0
1045,nan,3,36.00,0,2,12.18,0,1,1
1092,nan,3,28.67,0,0,15.50,0,0,1
986,nan,1,25.00,0,0,26.00,1,0,0


,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
count,0.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00
mean,nan,2.27,30.04,0.38,0.35,32.04,0.24,0.65,0.36
std,nan,0.84,12.75,0.60,0.72,40.70,0.43,0.48,0.48
min,nan,1.00,0.17,0.00,0.00,0.00,0.00,0.00,0.00
25%,nan,1.00,22.00,0.00,0.00,7.90,0.00,0.00,0.00
50%,nan,3.00,28.67,0.00,0.00,14.45,0.00,1.00,0.00
75%,nan,3.00,36.38,1.00,0.00,31.47,0.00,1.00,1.00
max,nan,3.00,71.24,2.00,3.00,169.73,1.00,1.00,1.00


### Fit a voting classifier
#### - selected based on the results in run_model.ipynb
#### - CV is increased from 5 to 10

In [10]:
BaseModels = [['Logistic', mdl.Logistic(fit_intercept=False, scale=False, C=10)]]
for i, j in itertools.product([5, 10, 20], ['uniform']):
    BaseModels += [['KNN_{}_{}'.format(i, j), mdl.KNN(scale=False, n_neighbors=i, weights=j)]]
for i, j, k in itertools.product([0.12, 0.06], [1., 0.8], [4, 8]):
    BaseModels += [['BoostedTree_learn{}_sub{}_leaf{}'.format(i, j, k),
                    mdl.BoostedTree(scale=False, n_estimators=256, loss='deviance', learning_rate=i, subsample=j,
                                    max_features=None, max_depth=None, max_leaf_nodes=k, random_state=1,
                                    balanceWeights=True)]]
for i in [8, 16, 32]:
    BaseModels += [['RandomForest_leaf{}'.format(i),
                    mdl.RandomForest(scale=False, n_estimators=256, max_features=None, max_depth=None,
                                     max_leaf_nodes=i, class_weight='balanced', random_state=1)]]
for i, j in itertools.product([2, 4], [0.06, 0.12, 0.25]):
    BaseModels += [['SVM_degree{}_gamma{}'.format(i, j),
                    mdl.SVM(scale=False, C=0.25, kernel='poly', degree=i, gamma=j, class_weight='balanced',
                            random_state=1)]]
for i in [0.25, 0.5, 1., 2.]:
    BaseModels += [['LinearLocal_bw{}'.format(i),
                    mdl.LogisticLinearLocal(scale=False, reg_type='ll', bw=i)]]
 
ModelVoteRegress = mdl.VoteRegress(cv=10, scale=True, Models=BaseModels, voting='soft', loss='devianceNormalized',
                                   baseClassifiersInfo=False)
ModelVoteRegress.fit(data)
ModelVoteRegress.predict(dataT)
ModelVoteRegress.printPlotSummary(cv=10)

****** VOTE REGRESS ******


-----Train and Test Sets-----

Sample Size (Train / Test): 891 / 418

Survived Rate (Train / Test): 0.38 / nan

-----Performance-----

...

SVM_degree2_gamma0.06 = 0.00

SVM_degree4_gamma0.12 = 0.00

SVM_degree4_gamma0.06 = 0.00

SVM_degree2_gamma0.25 = 0.00

<TEST OUTCOMES ARE NOT VALID: ROC PLOT IS NOT AVAILABLE>

### Write the submission file

In [11]:
submissionPath = os.path.join('data', 'data_derived', 'submission.csv')
submission = pd.DataFrame(index=dataT.index, columns=['Survived'])
submission.loc[:, 'Survived'] = ModelVoteRegress.ytH
submission.head()
submission.to_csv(submissionPath, index_label='PassengerId')